## Data Preparation stage

##### Join raw accident data with fault DB, update parameters

In [1]:
import pandas as pd
from tqdm import tqdm
from src.data.data_prep import convert_cols_type

In [2]:
df_fil = pd.read_csv('../data/interim/data_bivac_filtered.csv')
print(f"Data shape: {df_fil.shape}")
print(f"N accidents after filtering: {df_fil.accident_id.nunique()}")

Data shape: (18444, 11)
N accidents after filtering: 4217


In [3]:
df_fau = pd.read_excel('../data/raw/fault with param.xlsx')
print(f"Data shape: {df_fau.shape}")
print(f"N accidents after in Fault DB: {df_fau.accident_id.nunique()}")

Data shape: (6252, 9)
N accidents after in Fault DB: 1173


In [5]:
cat_features_fil = [
    'accident_id', 
    'vehicle_id', 
    'circumstances', 
    'road_signs',
    'road_surface', 
    'visibility', 
    'road_conditions',
    'address_id'
    ]

df_fil.accident_date = pd.to_datetime(df_fil.accident_date)
df_fil = convert_cols_type(
    df=df_fil,
    features_l=cat_features_fil,
    type_='str'
)

In [8]:
cat_features_fau = [
    'accident_id', 
    'accident_vehicle_id', 
    'circumstances', 
    'road_signs',
    'road_surface', 
    'visibility', 
    'road_conditions'
    ]

df_fau = convert_cols_type(
    df=df_fau,
    features_l=cat_features_fau,
    type_='str'
)

In [9]:
print(f"{df_fau.accident_id.nunique()} accidents in Fault DB")
print(f"All accidents from Fault DB are in our filtered data: {df_fil[df_fil.accident_id.isin(df_fau.accident_id.unique())].accident_id.unique() in df_fau.accident_id.unique()}")

1060 accidents in Fault DB
All accidents from Fault DB are in our filtered data: True


In [10]:
df_fau.type.value_counts()

FP    5246
FS     868
FA     138
Name: type, dtype: int64

In [11]:
df_fau.columns

Index(['accident_vehicle_id', 'accident_id', 'type', 'case', 'circumstances',
       'road_signs', 'road_surface', 'visibility', 'road_conditions'],
      dtype='object')

In [12]:
df_fau.pivot_table(['case',
                    ], ['accident_vehicle_id', 'accident_id', 'circumstances',
                    'road_signs',
                    'road_surface',
                    'visibility',
                    'road_conditions'], 'type', aggfunc='first')#.reset_index()

case  \
type                                                                                               FA   
accident_vehicle_id accident_id circumstances road_signs road_surface visibility road_conditions        
                    51632       12            nan        1            1          1                NaN   
                                3             nan        1            1          1                NaN   
                    51642       12            nan        1            1          1                NaN   
                                4             nan        1            1          1                NaN   
                    51651       12            nan        nan          nan        nan              NaN   
...                                                                                               ...   
99933               47288       35            163        1            1          1                NaN   
                                              31         1            1          1                NaN   
                                4             13         1            1          1                NaN   
                                              163        1            1          1                NaN   
                                              31         1            1          1                NaN   

                                                                                                      \
type                                                                                              FP   
accident_vehicle_id accident_id circumstances road_signs road_surface visibility road_conditions       
                    51632       12            nan        1            1          1                NG   
                                3             nan        1            1          1                NG   
                    51642       12            nan        1            1          1                FG   
                                4             nan        1            1          1                FG   
                    51651       12            nan        nan          nan        nan              NG   
...                                                                                               ..   
99933               47288       35            163        1            1          1                NG   
                                              31         1            1          1                NG   
                                4             13         1            1          1                NG   
                                              163        1            1          1                NG   
                                              31         1            1          1                NG   

                                                                                                       
type                                                                                               FS  
accident_vehicle_id accident_id circumstances road_signs road_surface visibility road_conditions       
                    51632       12            nan        1            1          1                NaN  
                                3             nan        1            1          1                NaN  
                    51642       12            nan        1            1          1                NaN  
                                4             nan        1            1          1                NaN  
                    51651       12            nan        nan          nan        nan              NaN  
...                                                                                               ...  
99933               47288       35            163        1            1          1                NaN  
                                              31         1            1          1                NaN  
                         

In [13]:
df_fau[df_fau.accident_id=='47319']

,accident_vehicle_id,accident_id,type,case,circumstances,road_signs,road_surface,visibility,road_conditions
3787,03,47319,FP,NG,3,nan,nan,nan,nan
3788,03,47319,FP,NG,12,nan,nan,nan,nan
3789,04,47319,FP,FG,2,nan,nan,nan,nan
3790,04,47319,FP,FG,11,nan,nan,nan,nan


In [14]:
df_fil[df_fil.accident_id=='47319']

,compensation_status,accident_id,vehicle_id,fd_decision,circumstances,road_signs,road_surface,visibility,road_conditions,accident_date,address_id
4713,VALIDATED_EIF,47319,74265,NG,3,nan,1,1,1,2020-05-31 12:11:00+04:00,6146
4714,VALIDATED_EIF,47319,74265,NG,12,nan,1,1,1,2020-05-31 12:11:00+04:00,6146
4715,VALIDATED_EIF,47319,74265,NG,25,nan,1,1,1,2020-05-31 12:11:00+04:00,6146
4716,VALIDATED_EIF,47319,74265,NG,,nan,1,1,1,2020-05-31 12:11:00+04:00,6146
4717,VALIDATED_EIF,47319,74263,FG,4,nan,1,1,1,2020-05-31 12:11:00+04:00,6146
4718,VALIDATED_EIF,47319,74263,FG,12,nan,1,1,1,2020-05-31 12:11:00+04:00,6146
4719,VALIDATED_EIF,47319,74263,FG,19,nan,1,1,1,2020-05-31 12:11:00+04:00,6146
4720,VALIDATED_EIF,47319,74263,FG,35,nan,1,1,1,2020-05-31 12:11:00+04:00,6146


In [15]:
df_fau

,accident_vehicle_id,accident_id,type,case,circumstances,road_signs,road_surface,visibility,road_conditions
0,34467,16141,FP,FG,2,nan,nan,nan,nan
1,34467,16141,FP,FG,12,nan,nan,nan,nan
2,34468,16141,FP,FG,2,nan,nan,nan,nan
3,34468,16141,FP,FG,12,nan,nan,nan,nan
4,34521,16169,FP,FG,2,nan,1,1,1
...,...,...,...,...,...,...,...,...,...
6247,1281,717,FP,FG,3,269,1,1,1
6248,1281,717,FP,FG,12,269,1,1,1
6249,1281,717,FP,FG,13,269,1,1,1
6250,128191,717,FP,NG,2,269,1,1,1


In [16]:
df = df_fil.copy()
# in the future df should be the dataframe with updated features from fault db and with the FINAL decision on fd_status

### Get categories names

In [17]:
cat_features = [
    'circumstances',
    'road_signs',
    'road_surface',
    'visibility',
    'road_conditions'
]

In [18]:
l_category_decoding = []
for sheet_name in tqdm(cat_features):
    df_temp = pd.read_excel('../data/raw/data with translations.xlsx', sheet_name=sheet_name)
    dict_temp = dict(zip(df_temp.iloc[:, 0].astype(str), df_temp.iloc[:, 1]))
    df[f"{sheet_name}_id"] = df[sheet_name]
    df[sheet_name] = df[f"{sheet_name}_id"].map(dict_temp)

    l_category_decoding.append({sheet_name: dict_temp})
    
print(df.shape)
df.head()

100%|██████████| 5/5 [00:09<00:00,  1.99s/it]

(18444, 16)


,compensation_status,accident_id,vehicle_id,fd_decision,circumstances,road_signs,road_surface,visibility,road_conditions,accident_date,address_id,circumstances_id,road_signs_id,road_surface_id,visibility_id,road_conditions_id
0,VALIDATED_EIF,16141,26245,NG,NaN,NaN,Asphalt-concrete,Clear night,Dry,2020-02-01 01:30:00+04:00,141147,nan,nan,1,2,1
1,VALIDATED_EIF,16141,26244,FG,First,NaN,Asphalt-concrete,Clear night,Dry,2020-02-01 01:30:00+04:00,141147,2,nan,1,2,1
2,VALIDATED_EIF,16141,26244,FG,Direct traffic,NaN,Asphalt-concrete,Clear night,Dry,2020-02-01 01:30:00+04:00,141147,12,nan,1,2,1
3,VALIDATED_EIF,16141,26244,FG,Green,NaN,Asphalt-concrete,Clear night,Dry,2020-02-01 01:30:00+04:00,141147,35,nan,1,2,1
4,VALIDATED_EIF,16169,26324,NG,First,NaN,Asphalt-concrete,Clear day,Wet,2020-02-01 18:00:00+04:00,141548,2,nan,1,1,2


In [19]:
import json
with open('../data/processed/feature_decoding.json', 'w') as f:
    json.dump(l_category_decoding, f)

### Feature Engineering

In [20]:
num_cardinality = 0
dummy_col_names = []
for feature in cat_features:
    l_unique_vals = df[feature].unique()
    n_unique_vals = len(l_unique_vals)
    l_range_nunique = [i for i in range(n_unique_vals)] # noqa

    num_cardinality += n_unique_vals
    l_ind_value_name = [f"{feature}_{i}" for i in l_unique_vals]
    dummy_col_names = dummy_col_names + l_ind_value_name


print(f"Cardinality: {num_cardinality}")

Cardinality: 63


In [36]:
# One hot encoding
df_prep = pd.get_dummies(df, columns=cat_features + ['fd_decision'])

# daytime features
df_prep['hour'] = df_prep.accident_date.dt.hour
df_prep['day_of_month'] = df_prep.accident_date.dt.day
df_prep['month'] = df_prep.accident_date.dt.month
df_prep['year'] = df_prep.accident_date.dt.year

# accident level features
df_temp = df[['accident_id', 'vehicle_id']].groupby('accident_id').agg({"vehicle_id": ["nunique"]}).reset_index()
df_temp.columns = ["".join(col).strip().replace("nunique", "") for col in df_temp.columns.values]
dict_acc_veh_count = dict(zip(df_temp.accident_id, df_temp.vehicle_id))

df_prep['n_vehicles_left_in_accident'] = df_prep.accident_id.map(dict_acc_veh_count)

cols_to_drop = [f"{col}_id" for col in cat_features] + \
               [
    'compensation_status',
    'address_id',
    'accident_date',
    'accident_id'
               ]


df_prep = df_prep.drop(cols_to_drop, axis=1).set_index('vehicle_id')
print(df_prep.shape)
df_prep.head()

(18444, 68)


,circumstances_Alarm signal,circumstances_Backward motion,circumstances_Beginning of traffic,circumstances_Breaking,circumstances_Detour,circumstances_Direct traffic,circumstances_Fifth,circumstances_First,circumstances_Fog lights,circumstances_Fourth,...,road_conditions_Other,road_conditions_Snow,road_conditions_Wet,fd_decision_FG,fd_decision_NG,hour,day_of_month,month,year,n_vehicles_left_in_accident
vehicle_id,,,,,,,,,,,,,,,,,,,,,
26245,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,1,2,2020,2
26244,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,1,1,2,2020,2
26244,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,1,1,2,2020,2
26244,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,2,2020,2
26324,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,1,18,1,2,2020,2


In [38]:
df_prep.to_csv('../data/processed/data_prepared.csv')